## AI based Chatbot

### Importing necessary libraries

In [1]:
import random
import json
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input
from tensorflow.keras.optimizers import SGD

### Loading the intents and preprocessing it

In [2]:
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('intents.json').read())
words=[]
classes=[]
documents=[]
ignore_letters=['?','!',',','.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))

### Converting text to numerical form to fit neural network model

In [3]:
training=[]
output_empty=[0]*len(classes)
for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])
random.shuffle(training)
training=np.array(training, dtype=object)
train_x=list(training[:,0])
train_y=list(training[:,1])

### Building and fitting the model

In [5]:
model=Sequential()
model.add(Input(shape=(len(train_x[0]),)))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size=5, verbose=1)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3461 - loss: 1.5751
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2909 - loss: 1.5949
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3223 - loss: 1.5139
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4764 - loss: 1.4573
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3001 - loss: 1.4194   
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5155 - loss: 1.2677
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3748 - loss: 1.3160
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4003 - loss: 1.2234
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5600 - loss: 1.1390
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5878 - loss: 1.0262
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6496 - loss: 0.9522
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7262 -

### Defining functions to perform various pre-processing tasks, predicting the class and displaying the response#

In [6]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)

def predict_class(sentence):
    bow=bag_of_words(sentence)
    res=model.predict(np.array([bow]))[0]
    error_threshold=0.25
    results=[[i,r] for i,r in enumerate(res) if r>error_threshold]
    results.sort(key=lambda x:x[1], reverse=True)
    return_list=[]
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

def get_response(intents,intents_json):
    tag=intents[0]['intent']
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
            result=random.choice(i['responses'])
            break
    return result

### Calling the functions post user input

In [ ]:
while True:
    message=input("")
    ints=predict_class(message)
    res=get_response(ints,intents)
    print(res)

 Hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Good to see you again!


 tell me about coding


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Programming, coding or software development, means writing computer code to automate tasks.


 where can i learn coding?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Start with the book named Let us C by Yashwant Kanetkar


 thank you


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Happy to help


 bye


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Goodbye!
